In [ ]:
# Importing Requried libraries
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.transforms.functional as FT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

from dataset import VOCDataset
from utils import non_max_suppression,cellboxes_to_boxes

In [2]:
#YOLO-V1 Neural Network
class Yolov1(nn.Module):
    def __init__(self, in_channels=3, split_size=7, num_boxes=2, num_classes=20):
        super(Yolov1, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            # Block 1
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            # nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2)),

            # Block 2
            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(192),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2)),

            # Block 3
            nn.Conv2d(192, 128, kernel_size=1),
            # nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, kernel_size=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2)),

            # Block 4
            nn.Conv2d(512, 256, kernel_size=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, kernel_size=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, kernel_size=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, kernel_size=1),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),

            nn.Conv2d(512, 512, kernel_size=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2)),

            # Block 5
            nn.Conv2d(1024, 512, kernel_size=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 512, kernel_size=1),
            # nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),

            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),

            # Block 6
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            # nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),

            # Block 7
            nn.Flatten(),         #flatten into 1D tensor
            nn.Linear(1024 * split_size**2, 400),
            nn.Dropout(0.0),
            nn.LeakyReLU(0.1),
            nn.Linear(400, split_size**2 * (num_classes + num_boxes*5))   #a linear layer which predicts the class probabilities and bounding boxes for each grid cell.
        )
      
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
mse = nn.MSELoss(reduction='sum') #mean squared error loss function

In [ ]:
# Loss: Coordinates of boxes
def loss_coord(pred, target, exists_box, better_box):
    pred = exists_box * (better_box * pred[..., 26:28] + (1-better_box) * pred[..., 21:23])
    target = exists_box * target[..., 21:23]
    return mse(pred, target)

In [ ]:
# Loss: Size of boxes
def loss_size(pred, target, exists_box, better_box):
    pred = exists_box * (better_box * pred[..., 28:30] + (1-better_box) * pred[..., 23:25])
    target = exists_box * target[..., 23:25]
    pred = torch.sign(pred) * torch.sqrt(torch.abs(pred) + 1e-6)
    target = torch.sqrt(target)
    return mse(pred, target)

In [ ]:
# Loss function: Prob of not containing object(Penalize both bounding boxes)
def loss_no_object(pred, target, exists_box, better_box):
    pred1 = (1-exists_box) * pred[..., 20:21]
    pred2 = (1-exists_box) * pred[..., 25:26]
    target = (1-exists_box) * target[..., 20:21]
    return mse(pred1, target) + mse(pred2, target)

In [ ]:
# Loss: Prob of each class
def loss_class(pred, target, exists_box, better_box):
    pred = exists_box * pred[..., :20]
    target = exists_box * target[..., :20]
    return mse(pred, target)

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, split_size=7, num_boxes=2, num_classes=20):
        super(YoloLoss, self).__init__()
        self.split = split_size
        self.boxes = num_boxes
        self.classes = num_classes
        self.lambda_coord = 5
        self.lambda_noobj = 0.5
    # Target -> ground truth; pred -> prdicted output 
    def forward(self, pred, target):
        pred = pred.reshape(-1, self.split, self.split, self.classes + self.boxes * 5)
        better_box = find_better_box(pred, target)
        exists_box = target[..., 20:21]
        coord_loss = loss_coord(pred, target, exists_box, better_box) # the loss for the bounding box coordinates
        size_loss = loss_size(pred, target, exists_box, better_box)   # the loss for the size of the selected box
        object_loss = loss_object(pred, target, exists_box, better_box) # the loss for the probability that an object exists in the cell
        no_object_loss = loss_no_object(pred, target, exists_box, better_box)
        class_loss = loss_class(pred, target, exists_box, better_box)   #the loss for the predicted probability distribution over object classes
        loss = self.lambda_coord * (coord_loss + size_loss) + object_loss + self.lambda_noobj * no_object_loss + class_loss
        return loss

In [ ]:
# Input a tensor of bounding boxes(x, y, w, h) and returns four tensors - left x, right x, top y, and bottom y coordinates of the boxes.
def get_margin_box(box):
    mid_x = box[..., 0:1]
    mid_y = box[..., 1:2]
    half_width = box[..., 2:3] / 2
    half_height = box[..., 3:4] / 2
    left_x = mid_x - half_width
    right_x = mid_x + half_width
    top_y = mid_y - half_height
    bot_y = mid_y + half_height
    return left_x, right_x, top_y, bot_y

# computes the IoU metric between the two boxes
def IoU(left_x1, right_x1, top_y1, bot_y1, left_x2, right_x2, top_y2, bot_y2):
    left_x_inte = torch.max(left_x1, left_x2)
    right_x_inte = torch.min(right_x1, right_x2)
    top_y_inte = torch.max(top_y1, top_y2)
    bot_y_inte = torch.min(bot_y1, bot_y2)
    intersection = (right_x_inte - left_x_inte).clamp(0) * (bot_y_inte - top_y_inte).clamp(0)
    box1_area = abs((right_x1 - left_x1) * (bot_y1 - top_y1))
    box2_area = abs((right_x2 - left_x2) * (bot_y2 - top_y2))
    union = box1_area + box2_area - intersection + 1e-6
    return intersection/union
    
# find better box based on iou metric
def find_better_box(pred, target):
    left_x1, right_x1, top_y1, bot_y1 = get_margin_box(pred[..., 21:25])
    left_x2, right_x2, top_y2, bot_y2 = get_margin_box(pred[..., 26:30])
    left_x_target, right_x_target, top_y_target, bot_y_target = get_margin_box(target[..., 21:25])
    
    iou1 = IoU(left_x1, right_x1, top_y1, bot_y1, left_x_target, right_x_target, top_y_target, bot_y_target)
    iou2 = IoU(left_x2, right_x2, top_y2, bot_y2, left_x_target, right_x_target, top_y_target, bot_y_target)
    ious = torch.cat([iou1.unsqueeze(3), iou2.unsqueeze(3)], dim=3)
    return torch.argmax(ious, dim=3)

In [ ]:
# takes in a list of transformations and applies them to both the input image and the target.
class JointCompose(object):
    def __init__(self, transforms):
        self.transforms = transforms
    def __call__(self, img, target):
        for t in self.transforms:
            img, target = t(img), target
        return img, target
    
transform = JointCompose([transforms.Resize((448, 448)), transforms.ToTensor(),])

In [ ]:
# Train function
# DEVICE = "cpu"
DEVICE = "cuda"  # Can switch to GPU by simply uncomment this line

def train(dataloader, model, optimizer, loss_fn):
    sum_loss = 0
    pbar = tqdm(dataloader, leave = False)
    for batch, (x, y) in enumerate(pbar):
        x, y = x.to(DEVICE), y.to(DEVICE)
        pred = model(x)
        loss = loss_fn(pred, y)
        sum_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return sum_loss / (batch+1)

In [ ]:
# Test function
def test(dataloader, model, optimizer, loss_fn):
    sum_loss = 0
    pbar = tqdm(dataloader, leave = False)
    with torch.no_grad():
        for batch, (x, y) in enumerate(pbar):
            x, y = x.to(DEVICE), y.to(DEVICE)
            pred = model(x)
            loss = loss_fn(pred, y)
            sum_loss += loss.item()
    return sum_loss / (batch+1)

In [ ]:
# Develop a function to plot the bounding box, predicted label and predicted probability
LABEL = ["airplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "dining table",
"dog", "horse", "motorbike", "person", "potted plant", "sheep", "sofa", "train", "TV monitor"]
LABEL_COLOR = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', 
               '#7f7f7f', '#bcbd22', '#17becf', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', 
               '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

def plot_image(image, boxes, ax):
    im = np.array(image)
    height, width, _ = im.shape
    ax.imshow(im)

    for box in boxes:
        obj = int(box[0])
        prob = box[1]
        prob = max(0, min(1, prob))

        topleft_x = box[2] - box[4] / 2
        topleft_y = box[3] - box[5] / 2
        rect = patches.Rectangle(
            (topleft_x * width, topleft_y * height),
            box[4] * width,
            box[5] * height,
            linewidth=2,
            edgecolor=LABEL_COLOR[obj],
            facecolor="none",
        )
        ax.annotate('{}: {:.2f}'.format(LABEL[obj], prob), 
                    (topleft_x * width+5, topleft_y * height-10), 
                    backgroundcolor=LABEL_COLOR[obj])
        ax.add_patch(rect)